In [1]:
import cv2
from imutils.face_utils import rect_to_bb
import dlib
import imutils
import os, time
import os.path
import numpy as np

In [2]:
mediaType = "image"  # image / video / webcamera
directory = "../face_age/"

cropPath = "output/"
imgType = "png"  # jpg, png

minFaceSize = (60, 60)
maxImageWidth = 1200

landmarksDB = "shape_predictor_68_face_landmarks.dat"
dlib_detectorRatio = 2
folderCharacter = "/"  # \\ is for windows

In [39]:
def chkEnv():
    if not os.path.exists(landmarksDB):
        print("There is no landmark db file for this path:", landmarksDB)

    if(mediaType=="image" and (not os.path.exists(directory))):
        print("There is no folder for this path:", imageFolder)

def getEyeShapes(landmarks):
    #right eye: 36~41
    #left eye: 42~47
    right_eyes = []
    left_eyes = []

    for id in range(36,42):
        right_eyes.append((landmarks.part(id).x, landmarks.part(id).y))

    for id in range(42,48):
        left_eyes.append((landmarks.part(id).x, landmarks.part(id).y))

    eyes_right_np = np.array(right_eyes)
    eyes_left_np = np.array(left_eyes)
    bbox_right = cv2.boundingRect(eyes_right_np)
    bbox_left = cv2.boundingRect(eyes_left_np)

    return bbox_left, bbox_right

def getNoseShapes(landmarks):
    #nose: 27~35
    nose = []

    for id in range(27,36):
        nose.append((landmarks.part(id).x, landmarks.part(id).y))

    nose_np = np.array(nose)
    bbox = cv2.boundingRect(nose_np)

    return bbox

def getMouthShapes(landmarks):
    #outer: 48~59
    #inner: 60~67
    outer_mouth = []
    inner_mouth = []

    for id in range(48,60):
        outer_mouth.append((landmarks.part(id).x, landmarks.part(id).y))

    for id in range(60,68):
        inner_mouth.append((landmarks.part(id).x, landmarks.part(id).y))

    outer_mouth_np = np.array(outer_mouth)
    inner_mouth_np = np.array(inner_mouth)
    bbox_outer_mouth = cv2.boundingRect(outer_mouth_np)
    bbox_inner_mouth = cv2.boundingRect(inner_mouth_np)

    return bbox_outer_mouth, bbox_inner_mouth

def labelFacial(img, dirNum, fname):
    ii = 0

    detector = dlib.get_frontal_face_detector()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector( gray , dlib_detectorRatio)

    if(len(rects)>0):
        BBOX_faces = []
        BBOX_leftEyebrow = []
        BBOX_rightEyebrow = []
        BBOX_leftEye = []
        BBOX_rightEye = []
        BBOX_nose = []
        BBOX_outer_mouth = []
        BBOX_inner_mouth = []

        ii = 0
        for faceid, rect in enumerate(rects):
            (x, y, w, h) = rect_to_bb(rect)
            if(w>minFaceSize[0] and h>minFaceSize[1]):
                BBOX_faces.append((x,y,w,h))

                shape = predictor(gray, rect)

                bbox_leftEye, bbox_rightEye = getEyeShapes(shape)
                BBOX_leftEye.append(bbox_leftEye)
                BBOX_rightEye.append(bbox_rightEye)

                bbox_nose = getNoseShapes(shape)
                BBOX_nose.append(bbox_nose)

                bbox_outer_mouth, bbox_inner_mouth = getMouthShapes(shape)
                BBOX_outer_mouth.append(bbox_outer_mouth)

                ii += 1

            if(ii>0):
                lefteye_crop = img[int(BBOX_leftEye[0][1]-0.5*BBOX_leftEye[0][3]):int(BBOX_leftEye[0][1]+1.5*BBOX_leftEye[0][3]),
                                  int(BBOX_leftEye[0][0]-0.5*BBOX_leftEye[0][2]):int(BBOX_leftEye[0][0]+1.5*BBOX_leftEye[0][2])]
        
                righteye_crop = img[int(BBOX_rightEye[0][1]-0.5*BBOX_rightEye[0][3]):int(BBOX_rightEye[0][1]+1.5*BBOX_rightEye[0][3]),
                                  int(BBOX_rightEye[0][0]-0.5*BBOX_rightEye[0][2]):int(BBOX_rightEye[0][0]+1.5*BBOX_rightEye[0][2])]
                
                nose_crop = img[int(BBOX_leftEye[0][1]):int(BBOX_nose[0][1]+BBOX_nose[0][3]),
                                  int(BBOX_nose[0][0]):int(BBOX_nose[0][0]+1.2*BBOX_nose[0][2])]
                
                mouth_crop = img[int(BBOX_outer_mouth[0][1]-0.2*BBOX_outer_mouth[0][3]):int(BBOX_outer_mouth[0][1]+1.2*BBOX_outer_mouth[0][3]),
                                  int(BBOX_outer_mouth[0][0]-0.2*BBOX_outer_mouth[0][2]):int(BBOX_outer_mouth[0][0]+1.2*BBOX_outer_mouth[0][2])]
                
                cv2.imwrite(cropPath+dirNum+fname+'_lefteye.'+imgType, lefteye_crop)
                cv2.imwrite(cropPath+dirNum+fname+'_righteye.'+imgType, righteye_crop)
                cv2.imwrite(cropPath+dirNum+fname+'_nose.'+imgType, nose_crop)
                cv2.imwrite(cropPath+dirNum+fname+'_mouth.'+imgType, mouth_crop)

In [4]:
chkEnv()

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(landmarksDB)

In [40]:
if(mediaType=="image"):
    for i, imageFolder in enumerate(os.listdir(directory)):
        if not os.path.isdir(cropPath + str(i+1) + '/'):
            os.mkdir(cropPath + str(i+1) + '/')
        fileDir = os.path.join(directory+imageFolder)
        for file in os.listdir(fileDir):
            filename, file_extension = os.path.splitext(file)
            if(file_extension == ".jpg" or file_extension==".jpeg" or file_extension==".png"):
                print("Processing: ",fileDir + folderCharacter + file)
                image = cv2.imread(fileDir + folderCharacter + file)
                if(image.shape[1] > maxImageWidth):
                    image = imutils.resize(image, width=maxImageWidth)
                labelFacial( image, str(i+1)+'/', file.replace(file_extension,'') )

Processing:  ../face_age/001/1000.png
Processing:  ../face_age/001/1018.png
Processing:  ../face_age/001/1030.png
Processing:  ../face_age/001/1062.png
Processing:  ../face_age/001/1068.png
Processing:  ../face_age/001/1075.png
Processing:  ../face_age/001/1077.png
Processing:  ../face_age/001/1081.png
Processing:  ../face_age/001/1087.png
Processing:  ../face_age/001/1089.png
Processing:  ../face_age/001/110.png
Processing:  ../face_age/001/1102.png
Processing:  ../face_age/001/1103.png
Processing:  ../face_age/001/1105.png
Processing:  ../face_age/001/1107.png
Processing:  ../face_age/001/1118.png
Processing:  ../face_age/001/1124.png
Processing:  ../face_age/001/1131.png
Processing:  ../face_age/001/1133.png
Processing:  ../face_age/001/1148.png
Processing:  ../face_age/001/1161.png
Processing:  ../face_age/001/1169.png
Processing:  ../face_age/001/1172.png
Processing:  ../face_age/001/1182.png
Processing:  ../face_age/001/1203.png
Processing:  ../face_age/001/1205.png
Processing:  

KeyboardInterrupt: 